In [17]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

df = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv', low_memory=False)



In [3]:
questions = df.iloc[0, :].T
questions

In [6]:
df = df.iloc[1:, :].reset_index(drop = True)
df.head()

In [25]:
countries = df.iloc[:, 3]
# countries에 df의 4번째 열('Q3')을 반환해준다
country_counts = (countries.value_counts().reset_index()).rename(columns = {'index':'Country','Q3': 'Count'})
#country_counts 에 countries의 횟수를 넣어주고 reset_index()) 한 후에 리네임으로 1행  =index 1열=Country , 2행 =Q3, 2열=Count 로 바꿔준다
country_counts['Country']=country_counts['Country'].replace('Viet Nam', 'Vietnam')
#country_counts의 Country에 있는 Viet Nam 을 .replace를 통해 Vietnam으로 바꿔주고 반환해준다. 
df.head()

from urllib.request import urlopen
#urllib 패키지를 통해 인터넷 소스를 가져 올 수 있게 한다.
import json
# 자바스크립트 기반으로 작성환 텍스트 데이터 포맷중 하나.
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
# urlopen을 통해 해당 웹페이지의 데이터를 .json 형식으로 받아온다.
info = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
# info에 지도를 그리기 위한 csv값을 반환해준다.
di = {}
# 딕셔너리 생성
for _,row in info.iterrows():
    di[row['COUNTRY']]=row['CODE']
 #info에 반환받은 각 국가의 CODE 값과 COUNTRY리를 새로운 di에 넣어준다.
count=0
n=[]
c=[]
code=[]
for key, value in di.items():
    for _,row in country_counts.iterrows():
        if key in row['Country']:
            count=count+1
            n.append(key)
            c.append(row['Count'])
            code.append(value)

dfc=pd.DataFrame()
dfc['Country']=n
dfc['Count']=c
dfc['Code']=code

dfcn = pd.pivot_table(dfc, index=['Country'],values=['Count'],aggfunc='sum').reset_index()
dfc.drop('Count', axis=1, inplace=True)
dfcn = pd.merge(left=dfcn, right=dfc, on='Country')

dfcn =dfcn.sort_values(by="Count")
dfcn.drop_duplicates(inplace=True)


In [26]:
 info.head()

In [21]:
fig = px.bar(dfcn, x='Country', y='Count', height=600, color='Count', color_continuous_scale='pinkyl', template='plotly_white')
fig.update_xaxes(type='category')
fig.update_layout(title={
        'text': "Country wise user count",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [19]:
fig = px.choropleth(dfcn, locations='Code', color='Count', color_continuous_scale='pinkyl',
                    hover_name="Country", template='plotly_white',
                    hover_data={
                        'Code':False
                    })
fig.update_layout(title={
        'text': "Demographic distribution of Users",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},dragmode=False)
fig.show()